In [1]:
import sqlalchemy
import pandas as pd
import server.model.connection as smc
import server.view.dataframes as smd

In [2]:
conn = smc.engine.connect()

In [5]:
sql = sqlalchemy.text(
    "with current AS (SELECT s.event_id as event_id, s.match, date from schedules sched, "
    "status s WHERE sched.event_id = s.event_id "
    "AND sched.match = s.match limit 1 ) "
    "SELECT * FROM current;"
)
pd.read_sql(sql, conn)

,event_id,match,date
0,25137,013-q,2018-02-17T10:48:00


In [ ]:
num_matches = 12
sql = sqlalchemy.text(
    "with current AS (SELECT status.event_id as event_id, status.match, date from schedules sched, "
    "status WHERE sched.event_id = status.event_id "
    "AND sched.match = status.match limit 1), "
    
    "recent_matches as ( SELECT * FROM ( "
    "SELECT row_number() over (partition by team order by sched.date desc) as r, "
    " sched.* from schedules sched, current c WHERE sched.event_id = c.event_id and sched.date <= c.date )"
    " row_schedule WHERE row_schedule.r <= " + str(
        num_matches) + " ORDER by team, date desc) "
    
    "SELECT * from recent_matches;"
)
pd.read_sql(sql, conn).head(20)

In [ ]:
select_sum = sqlalchemy.text(
    "with current AS (SELECT status.event_id as event_id, status.match, date from schedules sched, "
    "status WHERE sched.event_id = status.event_id "
    "AND sched.match = status.match limit 1), "
    
    "recent_matches as ( SELECT * FROM ( "
    "SELECT row_number() over (partition by team order by sched.date desc) as r, "
    " sched.* from schedules sched, current c WHERE sched.event_id = c.event_id and sched.date <= c.date )"
    " row_schedule WHERE row_schedule.r <= " + str(
        num_matches) + " ORDER by team, date desc), "
    
    "team_match_count as ( "
    "select team, count(team) as team_matches from recent_matches group by team"
       ") "
       
    "SELECT * FROM team_match_count;"
)
pd.read_sql(select_sum, conn).head()

In [ ]:
evt_id = 25137
sql = sqlalchemy.text(
    "with current AS (SELECT status.event_id as event_id, status.match, date from schedules sched, "
    "status WHERE sched.event_id = status.event_id "
    "AND sched.match = status.match limit 1), "
    
    "recent_matches as ( SELECT * FROM ( "
    "SELECT row_number() over (partition by team order by sched.date desc) as r, "
    " sched.* from schedules sched, current c WHERE sched.event_id = c.event_id and sched.date <= c.date )"
    " row_schedule WHERE row_schedule.r <= " + str(
        num_matches) + " ORDER by team, date desc), "
    
    "team_match_count as ( "
    "select team, count(team) as team_matches from recent_matches group by team"
       ") "
    
    "SELECT teams.name AS team, phases.name AS phase, tasks.name AS task, actors.name AS actor, "
    "MAX(team_match_count.team_matches) AS matches, "
    "SUM(successes) AS sum_successes, MAX(successes) as max_successes, MIN(successes) as min_successes, "
    "COUNT(successes) AS count_successes, AVG(successes) as avg_successes, "
    "SUM(attempts) AS sum_attempts, MAX(attempts) as max_attempts, MIN(attempts) AS min_attempts, "
    "COUNT(attempts) AS count_attempts, AVG(attempts) as avg_attempts, "
    "MAX(cycle_times) AS max_cycle_times, MIN(cycle_times) AS min_cycle_times, "
    "AVG(cycle_times), COUNT(cycle_times) AS count_cycle_times "
    "FROM ((((teams FULL OUTER JOIN measures ON teams.id=measures.team_id) "
    "LEFT JOIN tasks ON tasks.id = measures.task_id) "
    "LEFT JOIN phases ON phases.id = measures.phase_id) "
    "LEFT JOIN events ON events.id = measures.event_id) "
    "LEFT JOIN actors ON actors.id = measures.actor_id "
    "LEFT JOIN matches ON matches.id = measures.match_id "
    "LEFT JOIN team_match_count ON team_match_count.team = teams.name "
    "RIGHT JOIN recent_matches ON recent_matches.match = matches.name AND team_match_count.team = teams.name "
    "AND recent_matches.team = team_match_count.team "
    "WHERE events.id = " + str(evt_id) + " AND actors.name<> 'alliance' "
    "GROUP BY teams.name, tasks.name, phases.name, actors.name "
    "ORDER BY teams.name, phases.name, tasks.name, actors.name;"
)
df = pd.read_sql(sql, conn)
df

In [ ]:
rnk_df = smd.ranking_df(12).head()
rnk_df

In [ ]:
rnk_df["auto"]["robot"]["autoLine"]

In [ ]:
meas_df = smd.measure_summary_df(12)
meas_df.loc(axis=1)

In [ ]:
meas_df.xs("placeScale", level="task")

In [ ]:
smd.match_num_df()